Team members

*   cs23mtech11030 - Parasa Naga Venkata Pavan
*   cs23mtech11021 - Abhishree Gajanan Khangar
*   cs23mtech11032 - Pulicharla Punith Kumar
*   cs23mtech11003 - Apurba Saha






In [ ]:
import numpy as np
import pandas as pd
from sympy import *
import random

In [ ]:
#importing data from csv file
data = pd.read_csv('/content/4_case.csv', header=None)

global flg
z = data.loc[0]
z = z[:-1]
z = np.array(z)

c = data.loc[1]
c = c[:-1]
c = np.array(c)

b = data.iloc[2:, -1].values
b =np.array(b)

a = data.iloc[2:,:-1]
a = np.array(a)

m=len(a)
n=len(a[0])

b1_ = {}
for i in range(len(a)):
    b1_[tuple(a[i])] = b[i]

flg =False

In [ ]:
print("data \n",data)

data 
    0    1    2  3
0 -1  0.0  0.0  0
1  0  0.7  0.5  0
2  1 -2.0 -2.0  0
3  1 -1.0  1.0  1
4  1 -1.0 -1.0 -1
5  1 -1.0  0.0  0
6  1  0.0 -1.0  0
7 -1  0.0  0.0  1


In [ ]:
# z=np.array([0,0,-1])

In [ ]:
#Checking the feasility
#Step 1
def point_in_region(a,b,x):
    return a @ x <= b

def step1(a,b,z):
    check=point_in_region(a,b,z)
    for i in check:
        if not i:
            return False
    return True

In [ ]:
step1(a,b,z)

True

In [ ]:
#counting the tight rows

def no_of_tight(a,b,x):
    tight_count=0
    for i in range(len(a)):
        if round(a[i] @ x.T,4) ==round(b[i],4):
            tight_count+=1
    return tight_count

In [ ]:
#checking whether we are at a vertex

def check_vertex(a1):
    if len(a1)==0:
        return False
    if np.linalg.matrix_rank(a1)==n:
        return True
    else:
        return False

In [ ]:
#split the matrix based on tight rows and untight rows

def spliting_matrix(a,b,x):
    a1=[]
    a2=[]
    b1=[]
    b2=[]
    #print(a)
    for i in range(a.shape[0]):
        # print("%%%%%%%%%%%%%%%%%")
        # print(a[i])
        # print(b[i])
        # print(a[i]@x.T)
        # print("#############")
        if round(a[i] @ x.T,4) ==round(b[i],4):
            a1.append(a[i])
            b1.append(b[i])
        else:
            a2.append(a[i])
            b2.append(b[i])
    a1=np.array(a1)
    a2=np.array(a2)
    b1=np.array(b1)
    b2=np.array(b2)
    return a1,a2,b1,b2

In [ ]:
def replace_zero(ar):
  eps = 0.0000000000000001
  count = 0
  for i in ar:
    if(i == 0):
      i = eps
      ar[count] = i
    count+=1
  return ar

In [ ]:
#function to add small values in b vector
def add_small_values(b, value):
    result = b.copy()
    for i in range(len(result)):
        result[i] = float(result[i]) + value ** (i+1)
    return result

In [ ]:
#reaching a vertex
#Step 2
def step2(a,b,x):
    global flg
    b_dub=b
    cnt=0
    gamma=0.1
    tight_rows=no_of_tight(a,b,x)
    a1,a2,b1,b2=spliting_matrix(a,b,x)

    while not check_vertex(a1):
        cnt+=1
        if len(a1)==0:
            a1=np.array([]).reshape((0, n))

        u = np.array(Matrix(a1).nullspace()).astype(float)
        u = u.flatten()
        u = np.reshape(u, (-1, n))
        u_hat = u / np.linalg.norm(u,axis=1,keepdims=True)
        u_hat=u_hat[0]

        dino = (np.dot(a2 , u_hat))

        dino = replace_zero(dino)

        alpha  = (b2 -np.dot(a2,x.T) )/ dino.T
        # print("alpha",alpha)
        alpha = [i for i in alpha if abs(i)<= 1e+15 and abs(i)!=0]
        alpha_1 = []
        # removing -ve alphas
        for aa in alpha:
            if aa > 0:
                alpha_1.append(aa)

        # print("alpha_1",alpha_1)
        # taking min of alphas
        if len(alpha_1) == 0:
          alpha_1 = max(alpha)
        else:
          alpha_1 = min(alpha_1)
        # print("alpha_1_1",alpha_1)
        u_hat = u_hat.T
        x = x + alpha_1 * u_hat
        # print("x",x)
        a1,a2,b1,b2=spliting_matrix(a,b,x)
        if(cnt==100):
          # gamma=round(gamma+0.1,2)
          cnt=0
          # print("Gamma",gamma)
          tight_rows=no_of_tight(a,b,x)
          a1,a2,b1,b2=spliting_matrix(a,b,x)
          # print("after gamma a=",a)
          # print("after gamma b",b)
          # print("after gamma x",x)
          # print("after gamma c",)

    if(no_of_tight(a1,b1,x)>n):
      print("Handling Degeneration")
      t=np.array(b).astype(float)
      b=add_small_values(t,gamma)
      # print("Non deg",b)
      gamma=gamma/2
      b,x=step2(a,b,z)
      # print("new x",x)
      flg=True
      a1,a2,b1,b2=spliting_matrix(a,b,x)
    elif((no_of_tight(a1,b1,x)==n) and (np.linalg.matrix_rank(a1)==n) ):
      # print("a1",a1.shape)
      # print("b",b)
      return b,x
    # print("a1",a1.shape)
    # print("b",b)
    return b,x

In [ ]:
# step2(a,b,z)

In [ ]:
#checking whether current point is an optimum
def check_optimum(a1,c):
    # print("inside op")
    # print("a1",a1)
    # print("c",c)
    # print(a1.shape)
    # print(c.shape)
    # print("----------------")
    alphas=np.linalg.solve(a1.T, c)
    #print(alphas)
    for i in alphas:
        if i<0:
            return false
    return True

In [ ]:
#Function to get optimum point
#Step 3
def step3(a,b,x,c):
    global flg
    cnt = 0
    gamma=0.1
    # print("a=",a)
    # print("b",b)
    # print("x",x)
    # print("c",)
    a1,a2,b1,b2=spliting_matrix(a,b,x)
    # print("------------------------------------------")
    # print("a1",a1)
    # print("hello")
    # print("c",c)
    while not check_optimum(a1,c):
        # print("a1",a1)
        # print(a1.shape)
        # print(c.shape)
        if((np.linalg.matrix_rank(a1)>=n) and (no_of_tight(a1,b1,x)==n)):
          print("****************************************")
          print("Present Vertex point= ",x)
          a1i=np.linalg.inv(a1).T
          alphas=np.linalg.solve(a1.T, c)
          # print(alphas)
          # print(len(alphas))
          u=[]
          for i in range(len(alphas)):
              if alphas[i] < 0:
                  u=a1i[i]
                  break
          u=np.array(u)
          u=-1*u
          u=u.T
          dino = np.dot(a2 , u)

          dino = replace_zero(dino)

          alpha  = (b2 -np.dot(a2,x.T) )/ dino
          # print("--------------")
          # print("prev alpha",alpha)
          #alpha = alpha[0]
          alpha_1 = []
          # removing -ve alphas
          cnt = 0
          for aa in alpha:
              if aa > 0:
                  cnt+=1
                  alpha_1.append(aa)

          if(len(alpha_1) == 0):
            print("Unbounded result Optimum cannot be found !!")
            return []

          # taking min of alphas
          alpha_1 = min(alpha_1)
          u_hat = u.T
          # print("alpha_1",alpha_1)
          # print("prev x",x)
          x = x + alpha_1 * u_hat

          # print("x=",x)
        elif (no_of_tight(a1,b1,x)>n):
            print("Handling Degeneration")
            b=add_small_values(b,gamma)
            gamma=gamma/2
            x=step2(a,b,z)
            flg=True
        # print("b",b)
        a1,a2,b1,b2=spliting_matrix(a,b,x)
        # print("------------")
        # print("a1",a1)
        # print("------------")

    k_tuple = tuple(map(tuple, a1))
    for i in k_tuple:
      b1_final.append(b1_.get(i))
    if flg:
      # print("a1",a1)
      # print("b1",b1)
      #return step2(a1,b1_final,z)[1]
      # print("hhhhhhhhhhhhhhhhh")
      # print(a1)
      # print(b1_final)
      return np.linalg.solve(a1,b1_final)

    else:
      return x

In [ ]:
if step1(a,b,z):
  b1_final=[]
  step2_res = step2(a,b,z)
  res = step3(a,step2_res[0],step2_res[1],c)
  print("Optimum point:",res)
else:
  print("Given point is not feasible.")

Handling Degeneration
****************************************
Present Vertex point=  [-1.     -1.0001  0.9991]
****************************************
Present Vertex point=  [-1.     -1.0001  1.0099]
Unbounded result Optimum cannot be found !!
Optimum point: []
